In [1]:
import json, uuid, re, tempfile, os, copy, zipfile, shutil
from functools import reduce
from urllib.parse import urlparse
from arcgis import gis
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from arcgis.features import FeatureSet
from arcgis.gis import GIS
import requests
import json
import time

In [2]:
src_gis_portal_url = "https://ispgeospatial.com/portal"
src_gis_service_user = ""
src_gis_service_user_pwd = ""
source_gis = GIS(src_gis_portal_url, src_gis_service_user, src_gis_service_user_pwd, verify_cert=True, use_gen_token = True)

print("Successfully logged in target as: " + source_gis.properties.user.username)

target_gis = GIS('pro')
print("Successfully logged in source as: " + target_gis.properties.user.username)



Successfully logged in target as: siteadmin
Successfully logged in source as: Ross.Ma@aurecongroup.com


In [317]:
def get_photo_encode(url,token):
    payload = {'token':token, 'f': 'json'}
    response = requests.post(url, data = payload)
    if response.status_code == 200:
        return(response.json())
    else:
        print("Request failed with status code:", response.status_code)

In [3]:
source_item = source_gis.content.get("ade65c8326d24b63b65df84330b8e63d")

In [319]:
name_available = target_gis.content.is_service_name_available(service_name = "service_ad7a1ce394334565bf82e26ed68ba10b", service_type = 'featureService')
print(name_available)

True


In [320]:
if name_available == True:
    create_parameters = {
        "name": "service_ad7a1ce394334565bf82e26ed68ba10b",
        "description": "SW_PR_PropertyInspection_Form",
        "capabilities": "Create,Query,Editing,Update,Delete,Sync",#need to turn off the editing, update, Delete as final publication function
        "spatialReference": {"wkid": "4326"},
        "zDefault": 0,
        "properties": {
            "path": "",
            "description": "",
            "copyright": ""
        }
    }
    service_item = target_gis.content.create_service(name = "service_ad7a1ce394334565bf82e26ed68ba10b", 
                                                     service_type = 'featureService', 
                                                     create_params = create_parameters, 
                                                     tags = ['XForm', 'XForms'], 
                                                     snippet = "", 
                                                     item_properties = {'title':'SW_PR_PropertyInspection_Form','typeKeywords':"source-ade65c8326d24b63b65df84330b8e63d"},
                                                     item_id='ade65c8326d24b63b65df84330b8e63d'
                                                    )
else:
    print('this service is existing')

In [321]:
#src_flc = FeatureLayerCollection.fromitem(source_item)
empty_service_item = target_gis.content.get("ade65c8326d24b63b65df84330b8e63d")
trg_flc = FeatureLayerCollection.fromitem(empty_service_item)

for layer_item in source_item.layers:
    src_url = layer_item.url
    src_fl_id = src_url.split("/")[-1]
    trg_fl_id = "/" + str(src_fl_id)
    print(src_url)
    json_str = json.loads(f'{layer_item.properties}')
    del json_str['serviceItemId']
    #del json_str['sourceSpatialReference']
    temp_json = {'layers':[]}
    temp_json["layers"].append(json_str)
    print("adding definition")
    trg_flc.manager.add_to_definition(temp_json)
    time.sleep(10)
    print("done definition")
    new_fl = FeatureLayer(trg_flc.url + trg_fl_id, gis = target_gis)
    src_fl = FeatureLayer(src_url, gis = source_gis)
    src_fset = src_fl.query("1=1")
    src_fset_df = src_fset.sdf
    batch_size = 100
    batches = [group for _, group in src_fset_df.groupby(src_fset_df.index // batch_size)]
    start_time = datetime.datetime.now()
    print('Starting at {}'.format(str(start_time)))
    for i, batch_df in enumerate(batches):

        final_fs = FeatureSet.from_dataframe(batch_df)
        att_str = {"adds":[],"updates":[],"deletes":[]}

        for feat in final_fs.features:
            id_num = feat.attributes['objectid']
            global_id = feat.attributes['globalid']
            #print(global_id)
            att_list = src_fl.attachments.get_list(oid = id_num)
            #att_list = src_fl.attachments.search(object_ids = str(id_num),return_url=True)
            #print(att_list)
            if len(att_list)>0:
                for att in att_list:
                    #print(att)
                    #print("attid:"+str(att['attachmentid']))
                    if att['keywords']:
                        #print(att['keywords'])
                        url = src_url + "/" + str(id_num) + "/attachments/" + str(att['attachmentid'])
                        token = source_gis._con.token
                        photo_encode = get_photo_encode(url,token)['Attachment']
                        #print(url)
                        att_json = {
                            "parentGlobalId": global_id,
                            "globalId":att['globalid'],
                            "contentType":att['contentType'],
                            "data":photo_encode,
                            "name":att['name'],
                            "keywords":att['keywords'],
                            "attachmentid": att['attachmentid'],
                            "att_name":att['att_name']
                        }
                        att_str['adds'].append(att_json)
                        
        print("attachment number in this batch: " + str(len(att_str['adds'])))

        #print(final_fs)
        result = new_fl.edit_features(adds = final_fs,use_global_ids=True, rollback_on_failure=True,attachments = att_str)
    end_time = datetime.datetime.now()
    print('finished at {}'.format(str(end_time)))
    print('Duration: {}'.format(end_time - start_time))

https://ispgeospatial.com/server/rest/services/Hosted/service_ad7a1ce394334565bf82e26ed68ba10b/FeatureServer/0
adding definition
done definition
Starting at 2024-03-12 19:12:08.704103
attachment number in this batch: 115
finished at 2024-03-12 19:13:20.422859
Duration: 0:01:11.718756
